## neural transalte 번역

- 문장 > 단어들(토큰)     : 토큰나이징
- 단어 > 벡터(1차원 배열) : 워드 임베딩

In [96]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

나는 남자 다
나는 소년 다
나는 소녀 다
너는 남자 ?
너는 여자 ?

나는 : 0
다 : 1
? : 2
남자 : 3
여자 : 4
소년 : 5
소녀 : 6

i : 0 am : 1 a : 2 ? : 3 man : 4 woman : 5 boy : 6 girl : 7 are : 8 you : 9

In [97]:
x = [[0, 3, 1], [0, 5, 1], [0, 6, 1], [7, 3, 2], [7, 4, 2]]
y = [[0, 1, 2, 4], [0, 1, 2, 6], [0, 1, 2, 7], [8, 9, 4, 3], [8, 9, 5, 3]]
len(x), len(y)

(5, 5)

In [98]:
x = np.array(x)
y = np.array(y)
x.shape, y.shape, x.dtype, y.dtype

((5, 3), (5, 4), dtype('int32'), dtype('int32'))

In [99]:
embedding_size = 10 # 한 단어는 사이즈 10 백터로 변환
vocabulary_size = np.max(x) + 1 # 전체 단어 수
vocabulary_size_target = np.max(x) + 1 # 전체 단어 수
embedding_size, vocabulary_size, vocabulary_size_target

(10, 8, 8)

In [103]:
import tensorflow.contrib.slim as slim
tf.reset_default_graph()
word_embeddings = tf.get_variable('embed', shape=[vocabulary_size, embedding_size])
embedded = tf.nn.embedding_lookup(word_embeddings, x) #5, 3 > 5, 3, 5
cell_encoder = tf.nn.rnn_cell.GRUCell(100, name='enc') # 출력 100개
cell_encoder = tf.nn.rnn_cell.DropoutWrapper(cell_encoder, output_keep_prob=0.9)
outputs, last_state = tf.nn.dynamic_rnn(cell_encoder, embedded, dtype=tf.float32)
cell_decoder = tf.nn.rnn_cell.GRUCell(100, name='dec') # 출력 100개
# 5:다섯개의 문장, 4:문장의 단어수 단어를 ?개의 배열로 변환해라
zero_matrix = tf.zeros([5, 4, embedding_size])
outputs, last_state = tf.nn.dynamic_rnn(cell_decoder, zero_matrix, initial_state=last_state)
out = slim.fully_connected(outputs, 10) #영어 단어 총 10개
cost = tf.compat.v1.nn.sparse_softmax_cross_entropy_with_logits(logits=out, labels=y)
cost = tf.reduce_mean(cost)
train = tf.train.GradientDescentOptimizer(0.5).minimize(cost)


In [104]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())
for i in range(100):
    _, c = sess.run([train, cost])
    if i%10==0 : print(i, c)

0 2.306279
10 2.2513955
20 2.2010555
30 2.1364312
40 2.0718522
50 1.9667625
60 1.8508165
70 1.7715724
80 1.7170804
90 1.6695442


In [105]:
np.set_printoptions(precision=2, suppress=True)
print(sess.run(out))

[[[0.   0.   0.15 0.   0.   0.   0.   0.   0.03 0.  ]
  [0.   0.   0.83 0.   0.66 0.   0.   0.   0.   0.  ]
  [0.   0.   1.35 0.19 1.23 0.   0.   0.   0.   0.  ]
  [0.   0.   1.77 0.55 1.72 0.   0.   0.   0.   0.  ]]

 [[0.   0.   0.52 0.   0.   0.   0.   0.   0.   0.  ]
  [0.   0.   1.13 0.   0.   0.   0.   0.   0.   0.  ]
  [0.   0.   1.59 0.   0.   0.   0.   0.   0.   0.  ]
  [0.   0.   1.93 0.   0.35 0.   0.   0.   0.   0.  ]]

 [[0.   0.   0.31 0.   0.   0.   0.   0.   0.   0.  ]
  [0.   0.   0.95 0.   0.   0.   0.   0.   0.   0.  ]
  [0.   0.   1.45 0.   0.15 0.   0.   0.   0.   0.  ]
  [0.   0.   1.82 0.   0.57 0.   0.   0.   0.   0.  ]]

 [[0.   0.   0.   0.   0.91 0.   0.   0.   5.28 4.06]
  [0.   0.   0.   1.4  2.   0.   0.   0.   2.55 3.51]
  [0.   0.   0.   3.05 2.84 0.   0.   0.   0.   2.45]
  [0.   0.   0.46 4.33 3.45 0.   0.   0.   0.   1.21]]

 [[0.   0.   0.   0.   0.17 0.   0.   0.   5.06 3.82]
  [0.   0.   0.   1.03 1.15 0.   0.   0.   2.53 3.31]
  [0.   0.   0.   2.